# Fitting Description

## Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/digital_breakthrough/task_3

/content/drive/MyDrive/digital_breakthrough/task_3


In [4]:
!pip install -r requirements.txt

     |████████████████████████████████| 72 kB 811 kB/s 
     |████████████████████████████████| 72 kB 1.2 MB/s 
     |████████████████████████████████| 8.6 MB 13.4 MB/s 
     |████████████████████████████████| 2.6 MB 59.5 MB/s 
     |████████████████████████████████| 74 kB 4.0 MB/s 
     |████████████████████████████████| 37.1 MB 79 kB/s 
     |████████████████████████████████| 948 kB 65.8 MB/s 
     |████████████████████████████████| 428 kB 96.3 MB/s 
     |████████████████████████████████| 392 kB 82.4 MB/s 
     |████████████████████████████████| 56 kB 5.9 MB/s 
     |████████████████████████████████| 68 kB 8.9 MB/s 
     |████████████████████████████████| 112 kB 77.6 MB/s 
     |████████████████████████████████| 77 kB 8.7 MB/s 
     |████████████████████████████████| 56 kB 5.8 MB/s 
     |████████████████████████████████| 636 kB 76.7 MB/s 
     |████████████████████████████████| 895 kB 66.7 MB/s 
     |████████████████████████████████| 3.3 MB 55.6 MB/s 
     |███████████████████████

In [1]:
cd /content/drive/MyDrive/digital_breakthrough/task_3

/content/drive/MyDrive/digital_breakthrough/task_3


## Load Data

In [2]:
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
sys.path.append('.')
from definitions import DATA_PATH, TRAIN_IMAGES, DOWNLOADED_TRAIN_IMAGES
from airotica.utils import read_image

In [3]:
train = pd.read_csv(DATA_PATH / 'train.csv')
train_url_only = pd.read_csv(DATA_PATH / 'train_url_only.csv')
train_url_loaded_images = pd.read_csv(DATA_PATH / 'train_loaded_images.csv')
test = pd.read_csv(DATA_PATH / 'test.csv')
sample_submission = pd.read_csv(DATA_PATH / 'sample_submission.csv')

In [4]:
train_labels = train.typology.unique()
typology_to_label = dict(zip(sorted(train_labels), range(len(train_labels))))
label_to_typology = {v:k for k, v in typology_to_label.items()}

In [5]:
MAPPER = {0: 0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 
          7:7, 8:8, 9:9, 10:10, 11:12, 12:13, 13:14,
          -1: -1
          }

 ## FitImages

In [6]:
import logging
from airotica.airotica import detectron
from src.predict import prediction

%matplotlib inline
%load_ext autoreload
%autoreload 2

log = logging.getLogger(__name__)

{"asctime": "2021-08-04 19:59:20", "name": "tensorflow", "filename": "tpu_cluster_resolver.py", "levelname": "DEBUG", "message": "Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client."}
{"asctime": "2021-08-04 19:59:20", "name": "h5py._conv", "filename": "__init__.py", "levelname": "DEBUG", "message": "Creating converter from 7 to 5"}
{"asctime": "2021-08-04 19:59:20", "name": "h5py._conv", "filename": "__init__.py", "levelname": "DEBUG", "message": "Creating converter from 5 to 7"}
{"asctime": "2021-08-04 19:59:20", "name": "h5py._conv", "filename": "__init__.py", "levelname": "DEBUG", "message": "Creating converter from 7 to 5"}
{"asctime": "2021-08-04 19:59:20", "name": "h5py._conv", "filename": "__init__.py", "levelname": "DEBUG", "message": "Creating converter from 5 to 7"}
{"asctime": "2021-08-04 19:59:22", "name": "matplotlib.pyplot", "filename": "pyplot.py", "levelname": "DEBUG", "message": "Loaded backend

In [24]:
# IMPORT
import cv2
import random
import os
import yaml
import torch
import logging
import numpy as np
import pandas as pd
import albumentations as A

# FROM
from tqdm import tqdm
from torchvision import transforms as tfs
from pathlib import Path
from airotica.augmentations import ValidationAugmentations

from airotica.utils import (
    load_resnet,
    save_results,
    convert_dict_to_tuple
)
from definitions import ROOT_DIR


log = logging.getLogger(__name__)


def detectron(config_name,
              save_path: str,
              model_path: str,
              model_type: str,
              number_of_images=10,
              experiment=False,
              use_description=True,
              save_result=True,
              return_result=True):

    with open(ROOT_DIR / config_name) as f:
        data = yaml.safe_load(f)
    config = convert_dict_to_tuple(dictionary=data)
    # device = torch.device(device_name)
    # print(f'device: {device_name}')
    seed = config.dataset.seed
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

    os.environ['CUDA_VISIBLE_DEVICES'] = config.cuda_id
    device_name = 'cuda' if torch.cuda.is_available() else 'cpu'

    model = load_resnet(
        path=model_path,
        model_type=model_type,
        num_classes=config.dataset.num_of_classes,
        device=device_name
    )

    softmax_func = torch.nn.Softmax(dim=1)
    validation_augmentation = ValidationAugmentations(config=config)
    preprocess = tfs.Compose(
        [
            tfs.ToTensor(),
            tfs.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ]
    )
    test_df = pd.read_csv(config.path.input_path)
    test_image_path = Path(config.path.test_images_path)
    save_p = Path(save_path)
    len_ = len(test_df)
    if not experiment:
        scores = np.zeros((len_, 14), dtype=np.float32)
        for idx, row in tqdm(test_df.iterrows(), total=len_):
            if use_description:
                if not pd.isnull(row['description']):
                    scores[idx] = -1
                    continue

            guid = row['guid']
            path = str(test_image_path / f'{guid}.jpg')
            img = cv2.imread(path,
                             cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
            if img is None:
                print(f'None img {path}')
                scores[idx] = -1
                continue

            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            crop, _ = validation_augmentation(
                image=img,
                annotation=None
            )
            crop = preprocess(crop).unsqueeze(0)
            crop = crop.to(device_name)
            out = model(crop)
            out = softmax_func(out).squeeze().detach().cpu().numpy()
            scores[idx] = out

            if idx % 50 == 0 and idx > 0 and save_result:
                save_results(
                    scores=scores,
                    frame_paths=test_df.guid.values,
                    save_path=save_p
                )
        result_df = save_results(
            scores=scores,
            frame_paths=test_df.guid.values,
            save_path=save_p,
            save_result=save_result,
            return_result=return_result
        )
        return result_df


In [53]:
import yaml
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from src.preprocess import preprocess
from src.metrics import f1_score
from definitions import (
    DATA_PATH,
    DESCRIPTION_PATH,
    PREDICTION_CONFIGS_PATH
)


def prediction(config_name: str,
               agg='sum',
               test=None,
               cat=0,
               return_first=False):
    with open(PREDICTION_CONFIGS_PATH / config_name) as f:
        input_dict = yaml.safe_load(f)
    if test is None:
        test = pd.read_csv(DATA_PATH / 'test.csv')
    if cat == 0:
        categories = (
            pd.read_csv(
                DESCRIPTION_PATH / 'categories.csv')['category'].tolist()
        )
    else:
        categories = (
            pd.read_csv(
                DESCRIPTION_PATH / 'categories2.csv')['category'].tolist()
        )

    test_ = test[~(test.description.isna())]
    pb = []
    item_name = test_.description
    for k, v in input_dict.items():
        model = v['model']
        tokenizer = v['tokenizer']
        inp = preprocess(item_name,
                         tokenizer_path=tokenizer)
        mdl = load_model(model,
                         custom_objects={'f1_score': f1_score})
        p = mdl.predict(inp, batch_size=256, verbose=True)
        pb.append(p)
    if agg == 'mean':
        total_array = np.mean(np.array(pb), axis=0)
    elif agg == 'median':
        total_array = np.median(np.array(pb), axis=0)
    elif agg == 'sum':
        total_array = np.sum(np.array(pb), axis=0)
    else:
        raise ValueError(f'unknown agg: {agg}')
    df_pb = pd.DataFrame(total_array, columns=categories)
    if return_first:
        df_pb.index = test_.index
        return df_pb
    pred = df_pb.idxmax(axis=1)
    pred.index = test_.index
    return pred


In [113]:
images_prediction = detectron(config_name='config5.yml', 
                              save_path='./sub/predictv4.csv',
                              model_path='./EXPERIMENTS/full_res/model_0001_0.857586.pth',
                              model_type='resnext101_32x8d',
                              save_result=True,
                              return_result=True,
                              use_description=False
                              )

  1%|          | 10/1223 [00:00<00:29, 40.54it/s]

None img data/images/1dd23c5e-bf5d-45b7-9894-07bfdf91ae80.jpg
None img data/images/c3d3a4fc-c0bd-43e6-ac98-6c502bb7e793.jpg


  2%|▏         | 20/1223 [00:00<00:30, 39.80it/s]

None img data/images/7ac0e95c-2157-4373-95ba-f14cf9abd33c.jpg
None img data/images/93a61ff7-61d9-428f-91aa-fdf61e9a8ab1.jpg


  3%|▎         | 37/1223 [00:00<00:31, 37.09it/s]

...

 97%|█████████▋| 1188/1223 [00:58<00:01, 26.05it/s]

None img data/images/99e6cd07-43c8-4269-82b9-8233b4679b2d.jpg


 98%|█████████▊| 1202/1223 [00:58<00:00, 30.29it/s]

None img data/images/b97cba7b-1ec2-4bbe-b35a-bb2a32dee190.jpg
None img data/images/1b86090b-2357-4661-bed3-4c9127887265.jpg


100%|██████████| 1223/1223 [00:59<00:00, 20.59it/s]


In [114]:
pred = prediction('predict.yaml', return_first=True)

tokenizing:   0%|          | 0/676 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
tokenizing: 100%|██████████| 676/676 [00:00<00:00, 9181.92it/s]


3/3 [==============================] - 1s 70ms/step


tokenizing: 100%|██████████| 676/676 [00:00<00:00, 8599.00it/s]


3/3 [==============================] - 1s 68ms/step


In [115]:
for i, row in images_prediction.iterrows():
    mx = np.max(row[:14])
    if mx >= 0.9:
        images_prediction.at[i, 'label'] = MAPPER[np.argmax(row[:14])]
    else:
        images_prediction.at[i, 'label'] = -2

In [116]:
ts = images_prediction[(images_prediction.label == -2) & (images_prediction.sign_0 != -1)]

In [117]:
preds = pred / 2

In [118]:
for i, row in ts.iterrows():
    if i in preds.index:
        mx = np.max(preds.loc[i].values)
        if mx >= 0.9:
            images_prediction.at[i, 'label'] = np.argmax(row[:14])
        else:
            images_prediction.at[i, 'label'] = -3
    else:
        images_prediction.at[i, 'label'] = -3

In [120]:
me = images_prediction[images_prediction.label.isin([-2, -3])]
desc_me = preds[preds.index.isin(me.index)]

In [122]:
f_me = images_prediction[images_prediction.sign_0.isin([-1])]

In [123]:
images_prediction.loc[f_me.index, 'label'] = desc_me.loc[f_me.index].idxmax(axis=1).map(typology_to_label)

In [125]:
for i, row in images_prediction[images_prediction.label.isin([-2, -3])].iterrows():
    mx = np.max(row[:14])
    images_prediction.at[i, 'label'] = MAPPER[np.argmax(row[:14])]


In [17]:
# images_prediction.loc[:, ['label']] = images_prediction['sign_0'].map(MAPPER)
# images_prediction.loc[:, ['result']] = images_prediction['label'].map(label_to_typology)
# im_test = images_prediction[images_prediction.sign_0 != -1]

In [131]:
sample_submission.loc[:, 'typology'] = images_prediction['label'].map(label_to_typology)

In [132]:
sample_submission.to_csv('./sub/imbertver4.csv', index=False)

In [18]:
sample_submission.loc[pred.index, 'typology'] = pred.values
sample_submission.loc[im_test.index, 'typology'] = im_test.result

In [21]:
sample_submission.to_csv('./sub/imbertver2.csv', index=False)